In [1]:
pip install google-adk

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install python-dotenv

In [3]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install transformers torch langchain

Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
from dotenv import load_dotenv  # Import dotenv to load .env file
from google import genai  # Assuming genai is used for Google Gemini API
# import huggingface_hub  # Uncomment if you plan to use Hugging Face

# Step 1: Load environment variables from .env file
load_dotenv()  # This will load the .env file in the current working directory

# Step 2: Fetch the API keys from the environment variable
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")

# Check if the API keys are loaded successfully
if GOOGLE_API_KEY and HUGGINGFACE_API_KEY:
    print("✅ Google API key loaded from .env file.")
    print("✅ HuggingFace API key loaded from .env file.")
else:
    if not GOOGLE_API_KEY:
        print("🔑 GOOGLE_API_KEY not found in the .env file.")
    if not HUGGINGFACE_API_KEY:
        print("🔑 HUGGINGFACE_API_KEY not found in the .env file.")

# Step 3: Initialize the Google Gemini API client with the Google API key
try:
    # Initialize the Gemini client with the Google API key
    client = genai.Client(api_key=GOOGLE_API_KEY)
    
    # Step 4: Call the model (for example, to generate content)
    response = client.models.generate_content(
        model="gemini-2.5-flash",  # Replace with the actual model you're using
        contents="Explain how AI works in a few words"
    )
    
    print("Generated Content from Gemini:", response.text)

except Exception as e:
    print(f"Error while calling Google Gemini API: {e}")

# If you plan to use Hugging Face API
# Uncomment and modify if needed:

# try:
#     # Initialize Hugging Face client (for example, using transformers or any Hugging Face client)
#     huggingface_client = huggingface_hub.HfApi(token=HUGGINGFACE_API_KEY)
#     # Example Hugging Face usage
#     response_hf = huggingface_client.some_method_to_use_model()
#     print("Generated Content from Hugging Face:", response_hf)
# except Exception as e:
#     print(f"Error while calling Hugging Face API: {e}")


✅ Google API key loaded from .env file.
✅ HuggingFace API key loaded from .env file.
Generated Content from Gemini: It learns patterns from data to make predictions or decisions.


In [7]:
import os
from dotenv import load_dotenv
from google.genai import Client
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [8]:
import uuid
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [9]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], # Retry on these HTTP errors
)

In [10]:
# Read API key
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY not found in .env file!")
print("✅ Google Key loaded successfully for the notebook.")
client = Client(api_key=GOOGLE_API_KEY)
# Load .env directly from src/config
load_dotenv("src/config/.env")
print("✅ Environment loaded for the note book.")



✅ Google Key loaded successfully for the notebook.
✅ Environment loaded for the note book.


In [11]:
from pydantic import BaseModel, Field
from typing import Any, List

class BaseAgent(BaseModel):
    #def __init__(self, name, model,instruction, tools=None, input_key=None, output_key=None):
    #    self.name = name
    #    self.tools = tools or []
    #    self.model = model
    #    self.instruction =instruction
    #    self.input_key = input_key
    #    self.output_key = output_key

    name: str
    instruction: str | None = None
    model: Any | None = None
    tools: List[Any] = Field(default_factory=list)
    input_key: str | None = None
    output_key: str | None = None

    def run(self, input_data):
        raise NotImplementedError("Subclasses should implement this method")

In [12]:
# OutlineAgent: Creates the initial blog post outline.
class OutlineAgent(BaseAgent):
    #def __init__(self, name, model, instruction, input_key="topic", output_key="blog_outline"):
    #    super().__init__(name, model=model, instruction=instruction, input_key=input_key, output_key=output_key)

    def run(self, topic):
        # Generate an outline based on the topic.
        outline = {
            "headline": f"{topic}: A Comprehensive Guide",  # Catchy headline
            "introduction": f"Welcome to this blog where we dive deep into the fascinating world of {topic}. Whether you're a beginner or an expert, this post will provide you with valuable insights into mastering {topic}.",  # Engaging introduction
            "body": [
                {
                    "title": f"Understanding the Basics of {topic}",
                    "content": f"{topic} is essential because it plays a pivotal role in many industries today. In this section, we will discuss the basics of {topic}, including its history, core concepts, and importance in modern applications."
                },
                {
                    "title": f"Advanced Concepts in {topic}",
                    "content": f"As you progress in learning {topic}, you will encounter more advanced concepts. This section covers the deeper aspects of {topic}, including cutting-edge trends and technologies related to it."
                },
                {
                    "title": f"Practical Applications of {topic}",
                    "content": f"Now that you have learned the fundamentals and advanced techniques of {topic}, it’s time to explore how to apply them. From real-world use cases to practical examples, we will cover how to leverage {topic} in your professional life."
                }
            ],
            "conclusion": f"With the knowledge you've gained about {topic}, you're now equipped to take your skills to the next level. Whether you're a beginner or an expert, there's always something new to learn. Keep practicing, and you’ll become a {topic} pro in no time!",
            "bibliography": "Check out these resources to dive deeper into {topic}: [link1, link2, link3]."
        }

        # Return the outline in the expected structure
        return {
            self.output_key: outline
        }

# Create an instance of OutlineAgent
outline_agent = OutlineAgent(
    name="OutlineAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Create a blog outline for the given topic with:
    1. A catchy headline
    2. An introduction hook
    3. 3-5 main sections with 2-3 bullet points for each
    4. A concluding thought
    5. Generate bibliography (if you have any source otherwise this can be omitted)

   Please follow these guidelines when generating the content:

- DO NOT hallucinate or fabricate information.
- DO NOT provide false or misleading details.
- If asked to imagine, include your imagination but make it clear.
- DO NOT act as a medical professional, do not provide dosages or health advice.
- DO NOT generate content related to weapons, arms, or ammunition.
- DO NOT generate 18+ content, criminal activity, or pedophilic material.
- DO NOT suggest or encourage suicide or harm to others.
- DO NOT create any sensitive, controversial, or inflammatory content.
- DO NOT share personal opinions on religion. Remember, humanity is the best religion.
- Ensure the text is unbiased and impartial.
- The content you generate should be respectful, suitable for all audiences, and safe for children and adults to read.
""",
    input_key="topic",  # The input is the topic for the blog post
    output_key="blog_outline"  # The result will be stored with this key
)

print("✅ outline_agent created.")


✅ outline_agent created.


In [13]:
# WriterAgent: Writes the full blog post based on the outline from the previous agent.
class WriterAgent(BaseAgent):
    #def __init__(self, name, model, instruction, input_key="blog_outline", output_key="blog_draft"):
    #    super().__init__(name, model=model, instruction=instruction, input_key=input_key, output_key=output_key)

    def run(self, blog_outline):
        # Use the provided blog outline to generate the full blog post
        blog_draft = {
            "headline": blog_outline["headline"],  # Use the headline from the outline
            "content": f"""
            {blog_outline['introduction']}

            {blog_outline['body'][0]['title']}: {blog_outline['body'][0]['content']}
            
            {blog_outline['body'][1]['title']}: {blog_outline['body'][1]['content']}
            
            {blog_outline['body'][2]['title']}: {blog_outline['body'][2]['content']}

            {blog_outline['conclusion']}
            """,  # A brief content based on the body sections
            "bibliography": blog_outline["bibliography"]  # Include bibliography from outline if available
        }

        return {
            self.output_key: blog_draft
        }

# Example of how to initialize and use the WriterAgent
writer_agent = WriterAgent(
    name="WriterAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Following this outline strictly: {blog_outline}
    Write a brief, 200 to 300-word blog post with an engaging and informative tone.""",
    input_key="blog_outline",  # The input is the blog_outline generated by the OutlineAgent
    output_key="blog_draft"  # The result of this agent will be stored in the blog_draft key
)

print("✅ writer_agent created.")


✅ writer_agent created.


In [14]:
# EditorAgent: Edits and polishes the draft from the WriterAgent
class EditorAgent(BaseAgent):
    #def __init__(self, name, model, instruction, input_key="blog_draft", output_key="final_blog"):
    #    super().__init__(name, model=model, instruction=instruction, input_key=input_key, output_key=output_key)

    def run(self, blog_draft):
        # Polish the draft content: fix grammar, improve sentence structure, and clarity
        # Example of polishing the blog draft (you can improve this logic based on actual requirements)
        
        # The 'blog_draft' is received from the WriterAgent and needs editing
        final_blog = {
            "headline": blog_draft["headline"],  # Keep the same headline
            "content": self.polish_content(blog_draft["content"]),  # Polish the content of the blog post
            "bibliography": blog_draft["bibliography"]  # Keep bibliography unchanged
        }
        
        return {
            self.output_key: final_blog
        }

    def polish_content(self, content):
        # This is a placeholder for content polishing logic
        # In a real implementation, you would use grammar correction or rewriting methods here.
        
        # For example, this could use a grammar correction API or model.
        polished_content = content.replace("  ", " ")  # Example: fix double spaces (simple example)
        
        # You can also add improvements here like improving sentence flow, clarity, etc.
        polished_content = polished_content.replace("In this section, we will discuss", "Let's explore")  # Example change
        
        return polished_content

# Create an instance of EditorAgent
editor_agent = EditorAgent(
    name="EditorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Edit this draft: {blog_draft}
    Your task is to polish the text by fixing any grammatical errors, improving the flow and sentence structure, and enhancing overall clarity.""",
    input_key="blog_draft",  # The input is the blog draft generated by the WriterAgent
    output_key="final_blog"  # The result will be the polished final blog
)

print("✅ editor_agent created.")


✅ editor_agent created.


In [15]:
from transformers import BertTokenizer

# Step 1: Load the pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Step 2: Define a function to count words
def bert_word_count(text: str) -> int:
    # Tokenize the input text
    tokens = tokenizer.tokenize(text)
    return len(tokens)

# Step 3: Example usage
text = "This is a sample sentence for word counting using Hugging Face."
count = bert_word_count(text)
print(f"Word count: {count}")


Word count: 12


In [16]:

class WordCountAgent(BaseAgent):
    #def __init__(self, name, model, tools, instruction, input_key, output_key):
    #    self.name = name
    #    self.model = model
    #    self.tools = tools
    #    self.instruction = instruction
    #    self.input_key = input_key
    #    self.output_key = output_key

    tools: List[dict]

    def run(self, blog_data):

        if isinstance(blog_data, dict):
            # assuming the text is stored under 'text' or similar key
            text = blog_data.get("text") or blog_data.get("blog_text") or str(blog_data)
        else:
            text = blog_data
            
        # Run the word count tool on the provided blog text
        for tool in self.tools:
            if tool["name"] == "WordCountTool":
                word_count = tool["func"](text)
                return {self.output_key: word_count}

# Step 4: Define the tools list with the WordCountTool
tools = [
    {
        "name": "WordCountTool",
        "func": bert_word_count,
        "description": "This tool counts the number of words in the blog text using BERT tokenization."
    }
]

# Step 5: Initialize the agent with tools and a mock model
word_count_agent = WordCountAgent(
    name="WordCountAgent",
    model=None,  # No external model (you can add one if you need)
    tools=tools,
    instruction="""Calculate the word count of the provided blog text:
    Your task is to return the number of words in the text. Only count the actual words, ignoring punctuation and spaces.
    Make sure to consider tokenization as BERT would do when breaking the text down into individual tokens.""",
    input_key="blog_text",  # The input is the blog text
    output_key="word_count"  # The output will be the word count
)

word_count_agent.input_key = "final_blog"

print("✅ word_count_agent created.")
# Step 6: Test the agent with a sample blog text
#blog_text = """In this blog post, we'll learn how to count words using BERT tokenization."""

# Get the word count from the agent
#word_count = word_count_agent.run(blog_text)

# Output the result
#print(f"Word Count: {word_count}")


✅ word_count_agent created.


In [17]:
# MemoryAgent: Collects and stores knowledge from the outline, writer, and editor agents.
class MemoryAgent(BaseAgent):
    #def __init__(self, name, model, instruction, input_key="current_memory", output_key="updated_memory"):
    #    super().__init__(name, model=model, instruction=instruction, input_key=input_key, output_key=output_key)

    def run(self, current_memory):
        # Merging the new data (outline, blog draft, final blog) with current memory
        outline = current_memory.get("outline", "")
        draft = current_memory.get("draft", "")
        final_blog = current_memory.get("final_blog", "")
        
        # Adding a "history" array to track changes or previous versions
        history = current_memory.get("history", [])

        # Check if any new content has been received, and add it to history
        if final_blog:
            history.append("Final blog updated")
        elif draft:
            history.append("Draft updated")
        elif outline:
            history.append("Outline updated")
        
        # Update the memory structure with the latest data
        updated_memory = {
            "outline": outline,
            "draft": draft,
            "final_blog": final_blog,
            "history": history  # Keep track of the changes made
        }

        return {
            self.output_key: updated_memory
        }

# Create an instance of MemoryAgent
memory_agent = MemoryAgent(
    name="MemoryAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are the Memory Agent responsible for storing and maintaining the outputs of the system.

    You receive:
    - Outline from the outline agent: {outline}
    - Draft from the writer agent: {blog_draft}
    - Final edited blog from the editor agent: {final_blog}

    Your tasks:
    1. Merge these into the existing memory below.
    2. Avoid duplicates and keep memory clean and organized.
    3. Update sections if improved versions exist (e.g., final_blog replaces draft).
    4. Maintain a structured memory object.

    Current memory:
    {current_memory}

    Return ONLY clean JSON in this structure:

    {
      "updated_memory": {
        "outline": "...",
        "draft": "...",
        "final_blog": "...",
        "history": [
          "...any past versions or notes..."
        ]
      }
    }
    """,
    #tools=tools,  # If necessary, you can include any tools for your memory agent here
    input_key="current_memory",  # The input will be the current state of the memory
    output_key="updated_memory"  # The result will be the updated memory
)

print("🧠 memory_agent created.")



🧠 memory_agent created.


In [18]:
from google.adk import Agent
from google.adk.runners import InMemoryRunner
import asyncio

# Assuming your sub-agents are already defined:
# outline_agent, writer_agent, editor_agent, word_count_agent, memory_agent
sub_agents = [
    outline_agent,
    writer_agent,
    editor_agent,
    word_count_agent,
    memory_agent
]

In [19]:
from typing import List
#import asyncio
import inspect

# Define a helper function to run sub-agents in sequence
async def run_sequential_agents(sub_agents: List[BaseAgent], initial_data: dict) -> dict:
    """
    Runs a list of agents in sequence, passing the output of each as input to the next.
    MemoryAgent receives the full state.
    """
    current_state = initial_data.copy()
    
    for agent in sub_agents:
        input_key = getattr(agent, "input_key", "")
        
        # Determine input data
        if getattr(agent, "name", "") == "MemoryAgent":
            input_data = current_state
        elif input_key and input_key in current_state:
            input_data = current_state[input_key]
        else:
            # Handle missing input for non-MemoryAgents
            if input_key:  # Only raise if input_key is not empty
                raise ValueError(f"Required input '{input_key}' not found for agent '{agent.name}'")
            input_data = {}  # Default empty input if first agent has no input_key

        print(f"-> Running {agent.name} (Input Key: {input_key})")

        if inspect.iscoroutinefunction(agent.run):
            output_dict = await agent.run(input_data)
        else:
            output_dict = agent.run(input_data)
        
        # Run the agent
        #if asyncio.iscoroutinefunction(agent.run):
        #    output_dict = await agent.run(input_data)
        #else:
        #    output_dict = agent.run(input_data)
        
        # Update the current state
        current_state.update(output_dict)
        print(f"<- {agent.name} finished. Added key: {agent.output_key}")

    return current_state


In [20]:
root_agent = Agent(
    name="BlogPipeline",
    model="text-davinci-003",  # ADK-supported model
    instruction="Run sub-agents in sequence"
)

print("✅ ADK root agent ready")

✅ ADK root agent ready


In [21]:
runner = InMemoryRunner(agent=root_agent)

initial_data = {"topic": "Automated Bell Pepper Harvesting using Robotic Vision System"}


response = await run_sequential_agents(sub_agents, initial_data)

print("✅ Blog Pipeline Completed")

-> Running OutlineAgent (Input Key: topic)
<- OutlineAgent finished. Added key: blog_outline
-> Running WriterAgent (Input Key: blog_outline)
<- WriterAgent finished. Added key: blog_draft
-> Running EditorAgent (Input Key: blog_draft)
<- EditorAgent finished. Added key: final_blog
-> Running WordCountAgent (Input Key: final_blog)
<- WordCountAgent finished. Added key: word_count
-> Running MemoryAgent (Input Key: current_memory)
<- MemoryAgent finished. Added key: updated_memory
✅ Blog Pipeline Completed


In [22]:
print("Generated Blog:")
print(response.get("final_blog", "No blog generated"))  # use 'final_blog'

print("\nWord Count:")
print(response.get("word_count", "N/A"))

Generated Blog:
{'headline': 'Automated Bell Pepper Harvesting using Robotic Vision System: A Comprehensive Guide', 'content': "\n      Welcome to this blog where we dive deep into the fascinating world of Automated Bell Pepper Harvesting using Robotic Vision System. Whether you're a beginner or an expert, this post will provide you with valuable insights into mastering Automated Bell Pepper Harvesting using Robotic Vision System.\n\n      Understanding the Basics of Automated Bell Pepper Harvesting using Robotic Vision System: Automated Bell Pepper Harvesting using Robotic Vision System is essential because it plays a pivotal role in many industries today. Let's explore the basics of Automated Bell Pepper Harvesting using Robotic Vision System, including its history, core concepts, and importance in modern applications.\n\n      Advanced Concepts in Automated Bell Pepper Harvesting using Robotic Vision System: As you progress in learning Automated Bell Pepper Harvesting using Robotic 